<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/6_Federated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Overview: https://www.tensorflow.org/federated

Image classification tutorial: https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification

# Installation

In [1]:
!pip install --quiet tensorflow==2.3.0
!pip install --quiet tensorflow_federated==0.17.0
!pip install --quiet --upgrade nest_asyncio
# !pip install -q tfds-nightly

In [2]:
import nest_asyncio
nest_asyncio.apply()
%load_ext tensorboard

In [3]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd 
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys 
sys.path.append('/content/drive/MyDrive/Tesi_magistrale/Tesi_magistrale')
import mf

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Import data

I dati vengono importati e poi splittati in train e test. 

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [6]:
data = mf.feature_engineering(data)
# data = mf.feature_scaling(data)
data = pd.get_dummies(data)

In [7]:
# data = data[200000:500000]
col_name = mf.get_col(data)

In [14]:
data['isFraud'].to_numpy().shape

(590533,)

Smote: https://imbalanced-learn.org/stable/generated/imblearn.over_sampling.SMOTE.html

RandomUnderSampler: https://imbalanced-learn.org/stable/generated/imblearn.under_sampling.RandomUnderSampler.html

In [8]:
def sampling(data, col_name, frac_over = 0.1, frac_under = 0.25):
  from imblearn.over_sampling import SMOTE
  from imblearn.under_sampling import RandomUnderSampler 
  sm = SMOTE(sampling_strategy=frac_over, random_state=42)
  us = RandomUnderSampler(sampling_strategy=frac_under, random_state=42)
  y = data['isFraud']
  X = data.drop(columns = ['isFraud'])
  print('Random undersampling')
  X_us, y_us = us.fit_resample(X, y)
  print(X_us)
  print(X_us.shape)
  print(y_us)
  print(y_us.shape)
  y_us = np.ndarray(shape=(np.shape(y_us)[0],1), buffer = y_us)
  print(y_us.shape)
  print('Smote oversampling')
  X_sm, y_sm = sm.fit_resample(X_us, y_us)
  # del X, y
  # del X_sm, y_sm
  y_res = np.ndarray(shape=(np.shape(y_sm)[0],1), buffer = y_sm)
  del y_us
  print('concatenation')
  dataset = np.concatenate((X_us,y_res), axis = 1)
  del X_us, y_res
  col_name.append('isFraud')
  print('dataframe building')
  dataset =pd.DataFrame(data=dataset, columns=col_name)  
  return dataset


In [9]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.1)

In [10]:
train_datas = sampling(train_data, col_name)

Random undersampling
[[3.49089400e+06 1.32007650e+07 1.83955107e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.22654300e+06 5.67083600e+06 1.11782082e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.33958600e+06 8.70627100e+06 6.94332680e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [3.46451800e+06 1.23584190e+07 3.37224936e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.29159200e+06 7.54576900e+06 7.59304058e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.43507100e+06 1.14063960e+07 3.12332288e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
(92740, 1104)
[0. 0. 0. ... 1. 1. 1.]
(92740,)
(92740, 1)
Smote oversampling


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: ignored

In [ ]:
del data

# Convert data

Il dataset deve essere convertito in un tensore, con componenti (feature_vector, label). 

In [ ]:
def to_tensor(data, n_clients = 5):
  shuffled = data.sample(frac=1)
  result = np.array_split(shuffled, n_clients)  

  res = []

  new_res = []
  label = []

  for dataset in result:
    res.append(mf.feature_scaling(dataset))
  
  for subset in res:
    label.append(subset['isFraud'])
    new_res.append(subset.drop(columns = ['isFraud']).to_numpy())

  dataset = {}
  for i in range(n_clients):
    dataset[i] = tf.data.Dataset.from_tensor_slices((new_res[i], label[i]))
  return dataset

In [ ]:
dataset = to_tensor(train_datas)
test_set = to_tensor(test_data)

In [ ]:
del train_data, test_data

# Federated data

Si definisce la funzione di preprocessing del dataset, che serve a creare l'OrderedDict, su cui si andranno a creare le batch necessarie per il training del modello. 

In [ ]:
NUM_CLIENTS = 5
NUM_EPOCHS = 10
BATCH_SIZE = 250
SHUFFLE_BUFFER = 10
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(e1, e2):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x = tf.cast(e1, tf.float32),
        y = tf.cast(e2, tf.int32))
      #  x=tf.reshape(element, [-1, 1103]),
      #  y=tf.reshape(target, [-1, 1]))
        # element)

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

preprocessed_example_dataset = preprocess(dataset[0])

# sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
#                                      next(iter(preprocessed_example_dataset)))

# sample_batch

I dati federati sono una lista di dataset divisi per cliente. 

In [ ]:
def make_federated_data(dataset):
  federated = []
  for i in dataset:
    federated.append(preprocess(dataset[i]))
  return federated

federated_train_data = make_federated_data(dataset)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

In [ ]:
del dataset

# Model creation and training

Creazione della rete neurale che sarà trainata. 

Viene anche definita la model function, in cui si specifica il modello, il tipo di input, la loss e le metriche da utilizzare. 

Infine si costruisce il processo di averaging, specificando l'optimizer da usare, cioè SGD, e il learning rate del server e del client. 

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input

def create_keras_model():
  model = Sequential()
  model.add(Input(shape=(1103,)))  
  model.add(Dense(512, activation='relu')) 
  model.add(Dropout(0.5)) 
  model.add(Dense(256, activation='relu')) 
  model.add(Dropout(0.5))
  model.add(Dense(128, activation='relu')) 
  model.add(Dropout(0.5))
  model.add(Dense(24, activation='relu')) 
  model.add(Dropout(0.5))
  # model.add(Dense(4, activation='relu')) 
  # model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid'))
  return model

def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  soglia = 0.5
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[tf.keras.metrics.BinaryAccuracy(), 
               tf.keras.metrics.Recall(thresholds=soglia),
               tf.keras.metrics.Precision(thresholds=soglia)])
  
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,  
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.4), #0.4
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1)) #0.8

Training del modello. 

Gpu usage: https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=Y04m-jvKRDsJ

In [ ]:
NUM_ROUNDS = 51
with tf.device('/device:GPU:0'):
  state = iterative_process.initialize()
  state, metrics = iterative_process.next(state, federated_train_data)
  print('round  1, metrics={}'.format(metrics))
  for round_num in range(2, NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    print('round {:2d}, metrics={}'.format(round_num, metrics))

# Model evaluation

Evaluation del modello sui test data. 

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_fn)
train_metrics = evaluation(state.model, federated_train_data)

In [ ]:
federated_test_data = make_federated_data(test_set)

In [ ]:
test_metrics = evaluation(state.model, federated_test_data)
str(test_metrics)

Board di tensorboad, per visualizzare la loss e le metriche in modo interattivo. 

In [ ]:
logdir = "/tmp/logs/scalars/training/"
summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    for name, value in metrics['train'].items():
      tf.summary.scalar(name, value, step=round_num)

In [ ]:
!ls {logdir}
%tensorboard --logdir {logdir} --port=0